In [157]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
import library as lib
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from gensim.models import Word2Vec
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [347]:
from gensim.models import Word2Vec

In [5]:
df = pd.read_pickle('preprocessed_data.pkl')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
df.head()

,context,question,answers,plausible_answers,is_impossible,context_pos_lemma,question_pos_lemma,answers_pos_lemma,plausible_answers_pos_lemma,context_simple,question_simple,answers_simple,plausible_answers_simple
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, start, becoming, popular]","[late, 1990s]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[what, areas, beyonce, compete, growing]","[singing, dancing]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, leave, destinys, child, become...",[2003],
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, city, state, beyonce, grow]","[houston, texas]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, decade, beyonce, become, famous]","[late, 1990s]",


In [40]:
test_df = df[:100].copy()

In [41]:
test_df

,context,question,answers,plausible_answers,is_impossible,context_pos_lemma,question_pos_lemma,answers_pos_lemma,plausible_answers_pos_lemma,context_simple,question_simple,answers_simple,plausible_answers_simple
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, start, becoming, popular]","[late, 1990s]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[what, areas, beyonce, compete, growing]","[singing, dancing]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, leave, destinys, child, become...",[2003],
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, city, state, beyonce, grow]","[houston, texas]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, decade, beyonce, become, famous]","[late, 1990s]",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LeToya Luckett and Roberson became unhappy wit...,Who replaced Luckett and Roberson in Destiny's...,Farrah Franklin and Michelle Williams.,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(who, WP), (replace, VBD), (luckett, JJ), (ro...","[(farrah, NN), (franklin, NN), (michelle, NN),...",,"[letoya, luckett, roberson, became, unhappy, m...","[who, replaced, luckett, roberson, destinys, c...","[farrah, franklin, michelle, williams]",
96,LeToya Luckett and Roberson became unhappy wit...,Who was blamed for Luckett and Roberson leavin...,Beyoncé,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(who, WP), (blame, VBD), (luckett, JJ), (robe...","[(beyoncé, NN)]",,"[letoya, luckett, roberson, became, unhappy, m...","[who, blamed, luckett, roberson, leaving, dest...",[beyoncé],
97,LeToya Luckett and Roberson became unhappy wit...,Who helped Beyoncé overcome her depression dur...,her mother,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(who, WP), (help, VBD), (beyoncé, VB), (overc...","[(mother, NN)]",,"[letoya, luckett, roberson, became, unhappy, m...","[who, helped, beyoncé, overcome, depression, y...",[mother],
98,LeToya Luckett and Roberson became unhappy wit...,Which newest member was removed from Destiny's...,Farrah Franklin,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(which, WDT), (new, JJS), (member, NN), (remo...","[(farrah, NN), (franklin, NN)]",,"[letoya, luckett, roberson, became, unhappy, m...","[which, newest, member, removed, destinys, child]","[farrah, franklin]",


In [110]:
# generating n-grams, let's start with 3-grams; will have to generate a separate df for this
context_set = set([x for x in test_df.context])

In [111]:
context_set

{'A self-described "modern-day feminist", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny\'s Child, making her one of the best-selling music artists of all time. She has won 20 Grammy Awards and is the most nominated woman in the award\'s history. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011. Time listed her among the 100 most influential people in the world in 2013 and 2014. Forbes maga

In [127]:
def conc_str(lst):
    conc_str=''
    for i in lst:
        conc_str+=i[0]
        conc_str+=' '
    return conc_str
test_df['context_set_col'] = test_df.context_pos_lemma.apply(conc_str)

In [129]:
test_df.context_set_col

0     beyoncé giselle knowles-carter biːˈjɒnseɪ bee-...
1     beyoncé giselle knowles-carter biːˈjɒnseɪ bee-...
2     beyoncé giselle knowles-carter biːˈjɒnseɪ bee-...
3     beyoncé giselle knowles-carter biːˈjɒnseɪ bee-...
4     beyoncé giselle knowles-carter biːˈjɒnseɪ bee-...
                            ...                        
95    letoya luckett roberson become unhappy mathews...
96    letoya luckett roberson become unhappy mathews...
97    letoya luckett roberson become unhappy mathews...
98    letoya luckett roberson become unhappy mathews...
99    the remain band member record independent woma...
Name: context_set_col, Length: 100, dtype: object

In [130]:
context_list = set(test_df.context_set_col)

In [131]:
context_list

{'a self-described modern-day feminist beyoncé create song often characterize theme love relationship monogamy well female sexuality empowerment on stage dynamic highly choreographed performance lead critic hail one best entertainer contemporary popular music throughout career span 19 year sell 118 million record solo artist 60 million destiny child make one best-selling music artist time she 20 grammy award nominate woman award history the recording industry association america recognize top certify artist america 2000s decade in 2009 billboard name top radio song artist decade top female artist 2000s artist millennium 2011 time list among 100 influential people world 2013 2014 forbes magazine also list powerful female musician 2015 ',
 'at age eight beyoncé childhood friend kelly rowland meet latavia roberson audition all-girl entertainment group they place group three girl girl tyme rap danced talent show circuit houston after see group rb producer arne frager bring northern califor

In [152]:
def conc_str(lst):
    """Adds together the words from a list of tuples
    of preprocessed words and their POS tag"""
    conc_str=''
    for i in lst:
        conc_str+=i[0]
        conc_str+=' '
    return conc_str

def get_freq_dist(df, df_cols=['context_pos_lemma', 'question_pos_lemma'], drop_SET_col=True, topn=50):
    """returns a list of tuples containing a string and its frequency from within the 
    total corpus.
    """
    
    total_set_tokens=[]
    
    for col in df_cols:
        df[f'{col}_SET_TOKENS'] = df[col].apply(conc_str)

        tokens_set_list = set(df[f'{col}_SET_TOKENS'])

        
        for x in tokens_set_list:
            tokenized = word_tokenize(x)
            for y in tokenized:
                total_set_tokens.append(y)
        if drop_SET_col:
            df.drop(columns=[f'{col}_SET_TOKENS'], inplace=True)

    test_freqdist = FreqDist(total_set_tokens)
    return test_freqdist.most_common(topn)
    
    

In [155]:
test_df.head()

,context,question,answers,plausible_answers,is_impossible,context_pos_lemma,question_pos_lemma,answers_pos_lemma,plausible_answers_pos_lemma,context_simple,question_simple,answers_simple,plausible_answers_simple
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, start, becoming, popular]","[late, 1990s]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[what, areas, beyonce, compete, growing]","[singing, dancing]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, leave, destinys, child, become...",[2003],
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, city, state, beyonce, grow]","[houston, texas]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, decade, beyonce, become, famous]","[late, 1990s]",


In [344]:
# def get_tfidf(df, df_cols=['context_pos_lemma', 'question_pos_lemma'], drop_SET_col=True): 
#     """returns a list of tuples containing a string and its Term Frequency Inverse
#     Document Frequency score from within the total corpus, that corpus being the 
#     columns of the dataframe you've passed in. This function ASSUMES the values 
#     passed into it contain a TUPLE, where the 0th value is the token you're after. 
#     df - dataframe
#     df_cols - list; which columns are to be used as the corpus
#     drop_SET_col - boolean; the fn creates a column with just the tokens for each row; 
#     if true, removes the newly created column after it's been used;
#     """
#     corpus=[]
#     full_corpus=[]
    
#     for col in df_cols:
#         df[f'{col}_SET_DOCS'] = df[col].apply(conc_str)

#         tokens_set_list = set(df[f'{col}_SET_DOCS'])
        
        
# #         corpus=[]
#         for sublist in tokens_set_list:
#             doc=[]
#             doc = word_tokenize(sublist)
#             for w in doc:
#                 full_corpus.append(w)
#             corpus.append(doc)
            
            
#         if drop_SET_col:
#             df.drop(columns=[f'{col}_SET_DOCS'], inplace=True)
            
            

#     tfidf = TfidfVectorizer(stop_words=None, lowercase=False)
#     transformed_corpus = tfidf.fit_transform(full_corpus) 
    
#     return tfidf, transformed_corpus


def get_tf_df(token_list):
    tf_df = pd.DataFrame(index=set(token_list), columns=['tf'])
    for t in set(token_list):
        t_count=0
        for w in token_list:
            if t==w:                
                t_count+=1
        tf_df['tf'][t] = t_count/len(token_list)
    return tf_df



def get_idf_df(data, columns=['context_pos_lemma', 'question_pos_lemma']):
    
    corpus=[]
    for col in columns:
        doc=''
        for i in data[col]:
            for j in i:
                doc+=j[0]
                doc+=' '
                
        corpus.append(doc)
    cv = CountVectorizer()
    tfidf_transformer= TfidfTransformer(smooth_idf=True, use_idf=True)
    
    word_count_vector = cv.fit_transform(corpus)
    tfidf_transformer.fit(word_count_vector)
    
    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

    return df_idf


def get_tf_idf_col(data, columns=['context_pos_lemma', 'question_pos_lemma']):
    
    ndf=data.copy()
    
#     tfidf, transformed_corpus = get_tfidf(data, df_cols=columns, drop_SET_col=True)

    idf_df = get_idf_df(data, columns=columns)
    
#     tfdf = get_tf_df(tfidf_model=tfidf, transformed_corpus=transformed_corpus)
    
    for col in columns:
        ndf[f'{col}_tfidf'] = 0
        for i in ndf[col]:
        # here use row index as document index/ID and retrieve TFIDF score from the get_tf_df model
            new_col_term=[]
            tok_list = [x[0] for x in i]
            tf_df = get_tf_df(tok_list)
            for j in i:
                w = j[0]                
                w_pos = j[1]
                tf_score = tf_df['tf'][w]
                idf_score = idf_df['idf_weights'][w]
                print(w, tf_score, idf_score)
                w_tfidf_score = tf_score*idf_df['idf_weights'][w]
                new_col_tuple=(w, w_pos, w_tfidf_score)
                new_col_term.append(new_col_tuple)
                
                # can and will get round to adding the actual tfidf score per word per do
                # rather than just the sum 
                
            ndf.loc[ndf[col]==i, f'{col}_tfidf'] = new_col_term
            
    return ndf

def get_n_grams(data, columns = ['context_pos_lemma', 'question_pos_lemma'], n = 3, topn=50):
    corpus=[]
    for col in columns:
        doc=''
        for i in data[col]:
            for j in i:
                doc+=j[0]
                doc+=' '
                
        corpus.append(doc)
        
    
    vocab = set(corpus)
#     if n==3:                 
#         ngram_measures = nltk.collocations.TrigramAssocMeasures()
#         ngram_finder = TrigramCollocationFinder.from_words(corpus)
# #         corpus_scored = ngram_finder.score_ngrams()             
#     elif n==2:
#         ngram_measures = nltk.collocations.BigramAssocMeasures()
#         ngram_finder = BigramCollocationFinder.from_words(corpus)
# #         corpus_scored = ngram_finder.score_ngrams()             
#     elif n==4:
#         ngram_measures = nltk.collocations.QuadgramAssocMeasures()
#         ngram_finder = QuadgramCollocationFinder.from_words(corpus)
# #         corpus_scored = ngram_finder.score_ngrams()             

#     corpus_scored = ngram_finder.score_ngrams(ngram_measures.raw_freq)
    
#     ngram_measures = nltk.collocations.BigramAssocMeasures()
#     ngram_finder = BigramCollocationFinder.from_words(vocab)
#     corpus_scored = ngram_finder.score_ngrams(ngram_measures.raw_freq)
    
    
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigram_finder = BigramCollocationFinder.from_words(corpus)
    corpus_scored = bigram_finder.score_ngrams(bigram_measures.raw_freq)
    
    return corpus_scored
    


In [345]:
get_n_grams(test_df, n=2)

[(('beyoncé giselle knowles-carter biːˈjɒnseɪ bee-yon-say born september 4 1981 american singer songwriter record producer actress born raise houston texas perform various sing dance competition child rise fame late 1990s lead singer rb girl-group destiny child manage father mathew knowles group become one worlds best-selling girl group time their hiatus saw release beyoncés debut album dangerously love 2003 establish solo artist worldwide earn five grammy award feature billboard hot 100 number-one single crazy love baby boy beyoncé giselle knowles-carter biːˈjɒnseɪ bee-yon-say born september 4 1981 american singer songwriter record producer actress born raise houston texas perform various sing dance competition child rise fame late 1990s lead singer rb girl-group destiny child manage father mathew knowles group become one worlds best-selling girl group time their hiatus saw release beyoncés debut album dangerously love 2003 establish solo artist worldwide earn five grammy award featur

In [317]:
new_test = get_tf_idf_col(test_df)

beyoncé
beyoncé 0.013333333333333334 1.0
giselle
giselle 0.013333333333333334 1.4054651081081644
knowles-carter


KeyError: 'knowles-carter'

In [276]:
def get_tf_df(token_list):
    tf_df = pd.DataFrame(index=set(token_list), columns=['tf'])
    for t in set(token_list):
        t_count=0
        for w in token_list:
            if t==w:                
                t_count+=1
        tf_df['tf'][t] = t_count/len(token_list)
    return tf_df

tok =['cat', 'sat', 'on', 'cat' ]

get_tf_df(tok)

,tf
on,0.25
sat,0.25
cat,0.5


In [264]:


y = get_idf_df(test_df)
y.head()

,idf_weights
100,1.000000
118,1.405465
1516,1.405465
19,1.000000
1970s,1.405465


In [241]:
test_df.head()

,context,question,answers,plausible_answers,is_impossible,context_pos_lemma,question_pos_lemma,answers_pos_lemma,plausible_answers_pos_lemma,context_simple,question_simple,answers_simple,plausible_answers_simple
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, start, becoming, popular]","[late, 1990s]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[what, areas, beyonce, compete, growing]","[singing, dancing]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, leave, destinys, child, become...",[2003],
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, city, state, beyonce, grow]","[houston, texas]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, decade, beyonce, become, famous]","[late, 1990s]",


In [348]:
voc = lib.get_total_vocab(df, columns=['context_pos_lemma', 'question_pos_lemma'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [349]:
len(voc)

97884

In [243]:
first_tfidf, tf_corp = get_tfidf(test_df)


In [244]:
first_tfidf.get_feature_names()

['100',
 '118',
 '1516',
 '19',
 '1970s',
 '1980s',
 '1981',
 '1990',
 '1990s',
 '1995',
 '1996',
 '1997',
 '1999',
 '19th',
 '20',
 '200',
 '2000',
 '2000s',
 '2001',
 '2003',
 '2005',
 '2006',
 '2008',
 '2009',
 '2010',
 '2011',
 '2013',
 '2014',
 '2015',
 '43',
 '43rd',
 '60',
 '663000',
 'ability',
 'able',
 'acadian',
 'acclaim',
 'act',
 'actor',
 'actress',
 'african',
 'after',
 'age',
 'agreement',
 'aim',
 'album',
 'alief',
 'all',
 'alongside',
 'also',
 'alter',
 'america',
 'american',
 'among',
 'ancestry',
 'angel',
 'ann',
 'announce',
 'annual',
 'anything',
 'apartment',
 'appear',
 'arcadian',
 'area',
 'arne',
 'art',
 'artist',
 'association',
 'at',
 'atlanta',
 'attend',
 'audition',
 'award',
 'baby',
 'band',
 'base',
 'bday',
 'be',
 'bear',
 'beat',
 'beautiful',
 'become',
 'bedroom',
 'bee',
 'begin',
 'believe',
 'best',
 'beyincé',
 'beyonce',
 'beyonces',
 'beyoncé',
 'beyoncés',
 'bible',
 'bill',
 'billboard',
 'birth',
 'bizet',
 'biːˈjɒnseɪ',
 'blac

In [246]:
tfidf_scores=[]

for d in range(0,len(tfidf_scores)):
    feature_index = tf_corp[d,:].nonzero()[1]
    tfidf_score = zip(feature_index, [tf_corp[d, x] for x in feature_index])
    tfidf_scores.append(tfidf_score.__next__())

In [247]:
tfdf= pd.DataFrame(tf_corp.toarray(), columns=first_tfidf.get_feature_names())
tfdf = tfdf.T

In [248]:
tfdf.loc[: , 'totalscores'] = tfdf.sum(axis=1)

In [252]:
tfdf['totalscores']['bee']

0.6047411445197997

In [249]:
tfdf.head(100)

0    1    2    3    4    5    6    7    8    9   10   11   12  \
100         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
118         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1516        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
19          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1970s       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
birth       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bizet       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
biːˈjɒnseɪ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
black       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
blame       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             13   14   15   16   17   18   19   20   21   22   23   24   25  \
100         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
118         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1516        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
19          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1970s       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
birth       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bizet       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
biːˈjɒnseɪ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
black       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
blame       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             26   27   28   29   30   31   32   33   34   35   36   37   38  \
100         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
118         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1516        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
19          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1970s       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
birth       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bizet       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
biːˈjɒnseɪ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
black       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
blame       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             39   40   41   42        43   44   45   46   47   48   49   50  \
100         0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
118         0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1516        0.0  0.0  0.0  0.0  0.625306  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
19          0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1970s       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...   
birth       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bizet       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
biːˈjɒnseɪ  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
black       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
blame       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             51   52   53   54   55   56   57   58   59   60   61   62   63  \
100         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
118         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [239]:
tfdf.sort_index(ascending=False, by='totalscores')

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


0    1    2    3    4    5    6    7    8    9   10   11   12  \
beyoncé     1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
group       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
child       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
beyonce     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
what        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1516        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bee         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
yon         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
television  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
made        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             13   14   15   16   17   18   19   20   21   22   23   24   25  \
beyoncé     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
group       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
child       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
beyonce     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
what        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1516        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bee         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
yon         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
television  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
made        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             26   27   28   29   30   31   32   33   34   35   36   37   38  \
beyoncé     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
group       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
child       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
beyonce     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
what        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1516        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bee         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
yon         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
television  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
made        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             39   40   41   42        43   44   45   46   47   48   49   50  \
beyoncé     0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
group       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
child       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
beyonce     0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
what        0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...         ...  ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...   
1516        0.0  0.0  0.0  0.0  0.625306  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bee         0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
yon         0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
television  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
made        0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             51   52   53   54   55   56   57   58   59   60   61   62   63  \
beyoncé     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
group       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [240]:
tfdf[0]['beyoncé']

1.0

ok, so now that we know how to generate the tf-idf scores, we need to be able to include them into a new column. 